In [ ]:
#雷震日記
#有些字無法判斷
import xlrd
import time
import sys

from ckip import CKIPSegmenter, CKIPParser
segmenter = CKIPSegmenter('104753018', 'sayanouta')

file = "C:/Users/user/Desktop/雷震日記全表單-資訊處理-20160329.xls"
book = xlrd.open_workbook(file)

#print ("The number of worksheets is", book.nsheets)
#print ("Worksheet name(s):", book.sheet_names())
sh = book.sheet_by_index(0)
#print (sh.name, sh.nrows, sh.ncols)
'''print ("Cell D30 is", sh.cell_value(rowx=1, colx=0))
for rx in range(sh.nrows):
    #print (type(sh.row(rx)[0]))
    print (sh.cell_value(rowx=rx, colx=0))'''

#16 21 29 34 36 53 55 66 68
for rx in range(sh.nrows):
    if rx == 0:
        continue
    content = sh.cell_value(rowx=rx, colx=8)
    
    print (rx)
    
    lines = []
    temp = ''
    
    for word in content:
        temp = temp + word
        if word == '。' or word == '，':
            lines.append(temp)
            temp = ''
    
    for line in lines:
        try:
            segmented_result = segmenter.process(line)

            words = []

            if segmented_result['status_code'] != '0':
                    print ('Process Failed: ' + segmented_result['status'])
            else:
                for sentence in segmented_result['result']:
                    for term in sentence:
                        words.append(term['term']+'('+term['pos']+')')
                        '''if term['pos'] == 'COMMACATEGORY' or term['pos'] == 'PERIODCATEGORY' or (term['pos'] == 'QUESTIONCATEGORY' and 
                            term['pos'] == '?'):
                            #print (' '.join(words))
                            #print ()
                            words = []'''
                print (' '.join(words))
        except:
            e = sys.exc_info()[0]
            print (rx,e)
    break

In [ ]:
#XML排版
from xml.etree import ElementTree as etree
from xml.etree.ElementTree import Element, SubElement, ElementTree
 
#title author bookname publisher publishdate page theme keyword text

root = Element('root')

article = SubElement(root, 'acticle')
article.set("no", "1")

title = SubElement(article, 'title')
author = SubElement(article, 'author')
bookname = SubElement(article, 'bookname')
publisher = SubElement(article, 'publisher')
publishdate = SubElement(article, 'publishdate')
page = SubElement(article, 'page')
theme= SubElement(article, 'theme')
keyword = SubElement(article, 'keyword')
text = SubElement(article, 'text')
text.text = '雷震日記'

tree = ElementTree(root)

#tree.write('C:/Users/user/Desktop/result.xml', encoding='utf-8')

xml_string = etree.tostring(root,'utf-8')
print (xml_string.decode('utf8'))

In [ ]:
#建立雷震日記XML
from xml.etree import ElementTree as etree
from xml.etree.ElementTree import Element, SubElement, ElementTree
import xlrd
import time
import sys
import datetime

from ckip import CKIPSegmenter, CKIPParser
segmenter = CKIPSegmenter('104753018', 'sayanouta')

file = "C:/Users/user/Desktop/雷震日記全表單-資訊處理-20160329.xls"
output_file = "C:/Users/user/Desktop/new_diary/diary_{}.xml"
book = xlrd.open_workbook(file)
sh = book.sheet_by_index(0)
 
#title author bookname publisher publishdate page theme keyword text

li = ['title','author','bookname','publisher','publishdate','page','theme','keyword']

root = Element('root')

ci = 0
fail = False

for rx in range(sh.nrows-ci):
    #rx = rx + ci
    if rx == 0:
        continue
    
    article = SubElement(root, 'acticle')
    article.set("no", str(rx))
    
    for i in range(len(li)):
        temp = SubElement(article,li[i])
        if li[i] == 'publishdate':
            a1_as_datetime = datetime.date(*xlrd.xldate_as_tuple(sh.cell_value(rowx=rx, colx=i), 0)[:3])
            temp.text = str(a1_as_datetime)
        else:
            temp.text = str(sh.cell_value(rowx=rx, colx=i))
    text = SubElement(article,'text')
    
    content = sh.cell_value(rowx=rx, colx=8)
    
    print (rx)
    
    lines = []
    temp = ''
    
    #文章終句例外判斷?
    for word in content:
        temp = temp + word
        if word == '。' or word == '，':
            lines.append(temp.strip())
            temp = ''
    
    for line in lines:
        try:
            segmented_result = segmenter.process(line)

            words = []
            sentence = SubElement(text, 'sentence')
            check = False

            if segmented_result['status_code'] != '0':
                    print ('Process Failed: ' + segmented_result['status'])
            else:
                for sentences in segmented_result['result']:
                    for term in sentences:
                        if term['pos'] == 'QUESTIONCATEGORY':
                            check = True
                        words.append(term['term']+'('+term['pos']+')')
                
                if check:
                    sentence.text = str(line)
                    sentence.set('error','古字')
                else:
                    sentence.text = ' '.join(words)
                
        except:
            sentence.text = str(line)
            sentence.set('error','斷詞失敗')
            e = sys.exc_info()[0]
            print (rx,e)
            #fail = True
    
    if rx % 10 == 0:
        '''if fail:
            root = Element('root')
            rx = rx - 10
            print ('return')
            fail = False
            continue'''
        tree = ElementTree(root)
        num = str(int(rx / 10))
        if len(num) == 1:
            num = '00'+num
        elif len(num) == 2:
            num = '0'+num
        tree.write(output_file.format(num), encoding='utf-8')
        root = Element('root')

tree = ElementTree(root)
tree.write(output_file.format('945'), encoding='utf-8')

#xml_string = etree.tostring(root,'utf-8')
#print (xml_string.decode('utf8'))
print ('end')

In [16]:
#問題句子所在比例分析
import xml.etree.ElementTree as ET
import os
import time

file_path = "C:/Users/user/Desktop/temp/"
file_list = []

for file in os.listdir(file_path):
    file_list.append(file)

all_num = 0
all_error_word = 0  
all_error_split = 0

for file in file_list:
    tree = ET.parse(file_path+file)
    root = tree.getroot()
    
    num = 0
    error_word = 0
    error_split = 0

    
    for child in root:
        for sentence in child[8]:
            num = num + 1
            if len(sentence.attrib) > 0:
                if sentence.attrib['error'] == '古字':
                    error_word = error_word + 1
                elif sentence.attrib['error'] == '斷詞失敗':
                    error_split = error_split + 1
                else:
                    print ('other')
    error_count = (error_word+error_split)*100/num
    print (file,num,error_word,error_split,str(round(error_count,2))+'%')
    
    all_num = all_num + num
    all_error_word = all_error_word + error_word
    all_error_split = all_error_split + error_split
    
print (all_num,all_error_word,all_error_split,str(round((all_error_split+all_error_word)*100/all_num,2))+'%')

diary_001.xml 189 41 0 21.69%
diary_002.xml 315 54 0 17.14%
diary_003.xml 331 49 2 15.41%
diary_004.xml 281 54 1 19.57%
diary_005.xml 223 39 0 17.49%
diary_006.xml 322 72 0 22.36%
diary_007.xml 329 60 0 18.24%
diary_008.xml 223 32 0 14.35%
diary_009.xml 259 40 0 15.44%
diary_010.xml 204 37 0 18.14%
diary_011.xml 207 48 0 23.19%
diary_012.xml 288 54 0 18.75%
diary_013.xml 306 51 1 16.99%
diary_014.xml 278 36 0 12.95%
diary_015.xml 436 48 1 11.24%
diary_016.xml 220 29 0 13.18%
diary_017.xml 223 34 1 15.7%
diary_018.xml 418 65 1 15.79%
diary_019.xml 500 91 2 18.6%
diary_020.xml 265 33 0 12.45%
diary_021.xml 326 56 0 17.18%
diary_022.xml 292 33 0 11.3%
diary_023.xml 257 45 0 17.51%
diary_024.xml 436 70 0 16.06%
diary_025.xml 260 33 0 12.69%
diary_026.xml 444 78 0 17.57%
diary_027.xml 458 59 1 13.1%
diary_028.xml 518 71 0 13.71%
diary_029.xml 452 76 1 17.04%
diary_030.xml 439 59 0 13.44%
diary_031.xml 476 81 0 17.02%
diary_032.xml 390 65 2 17.18%
diary_033.xml 317 55 0 17.35%
diary_034.xml 

In [7]:
#diary XML內容抓取 one 情態動詞
import codecs
import os
import xml.etree.ElementTree as ET
import time
import sys

modalverb = ["應","要","可","能","可以","須","應該","必須","會","得","需要","當","應當","能夠","該","需"]
file_path = '../../desktop/temp/'

file_list = []

for file in os.listdir(file_path):
    file_list.append(file)

one_verb = {} 
    
error_num = 0
skip = 0

for file in file_list:
    
    tree = ET.parse(file_path+file)
    root = tree.getroot()

    for sentence in root.iter('sentence'):
        if len(sentence.attrib) > 0:
            skip = skip + 1
            continue
            
        try:
            temp = sentence.text.split()

            verb_list = []
            
            for i in temp:
                word = i.split('(')
                word[1] = '('+word[1].split('[')[0]

                if word[0] in modalverb:
                    verb_list.append(''.join(word))

            if len(verb_list) > 0:
                for l in verb_list:
                    if l in one_verb:
                        one_verb[l] = one_verb[l] + 1
                    else:
                        one_verb[l] = 1
     
        except:
            '''e = sys.exc_info()[0]
            print (e)
            time.sleep(0.3)'''
            error_num = error_num + 1

one_verb_count = 0

for i in one_verb:
    one_verb_count = one_verb_count + one_verb[i]

print (error_num) #0
print (len(one_verb)) #37
print (one_verb_count) #14964
print (skip) #18031

one_verb = sorted(one_verb.items(), key=lambda d:d[1], reverse = True)

for i in one_verb:
    print (i[0],i[1])

0
37
14964
18031
要(ADV) 4474
可(ADV) 2178
得(T) 1628
可以(ADV) 1015
應(ADV) 946
會(ADV) 897
該(DET) 603
能(ADV) 537
要(Vt) 403
當(P) 352
必須(ADV) 338
須(ADV) 320
會(N) 209
應該(ADV) 209
得(Vt) 176
該(ADV) 161
得(ADV) 91
需要(Vt) 88
需(Vt) 55
當(Vt) 48
當(ADV) 36
可(Vi) 36
能夠(ADV) 35
需(ADV) 27
應(P) 21
能(Vt) 17
可以(Vi) 17
應該(Vi) 12
需要(N) 11
會(Vt) 7
應當(ADV) 6
應(Vt) 5
要(Nv) 2
能(N) 1
需(Nv) 1
當(N) 1
得(ASP) 1


In [2]:
#diary XML內容抓取 pair情態動詞
import codecs
import os
import xml.etree.ElementTree as ET
import time
import sys

modalverb = ["應","要","可","能","可以","須","應該","必須","會","得","需要","當","應當","能夠","該","需"]
file_path = 'C:\\Users\\user\\Desktop\\SNA\\meeting\\source\\diary\\'

file_list = []

for file in os.listdir(file_path):
    file_list.append(file)

two_verb = {} 
    
error_num = 0
much = 0
skip = 0
index_line = 0

for file in file_list:
    
    tree = ET.parse(file_path+file)
    root = tree.getroot()

    for sentence in root.iter('sentence'):
        if len(sentence.attrib) > 0:
            skip = skip + 1
            continue
        
        index_line = index_line + 1
            
        try:
            temp = sentence.text.split()

            verb_list = []
            
            for i in temp:
                word = i.split('(')
                word[1] = '('+word[1].split('[')[0]

                if word[0] in modalverb:
                    verb_list.append(''.join(word))

            if len(verb_list) >= 2:
                if len(verb_list) >= 3:
                    much = much + 1
                for l in range(len(verb_list)-1):
                    k = l + 1
                    if (verb_list[l],verb_list[k]) in two_verb:
                        two_verb[(verb_list[l],verb_list[k])] = two_verb[(verb_list[l],verb_list[k])] + 1
                    else:
                        two_verb[(verb_list[l],verb_list[k])] = 1
     
        except:
            '''e = sys.exc_info()[0]
            print (e)
            time.sleep(0.3)'''
            error_num = error_num + 1

pop = []

#是否照順序
'''for i in two_verb:
    if (i[1],i[0]) in two_verb and i not in pop and i[0] != i[1]:
        two_verb[i] = two_verb[i] + two_verb[(i[1],i[0])]
        pop.append((i[1],i[0]))

print (len(two_verb)) #230/614
print (len(pop)) #101/188
for j in pop:
    two_verb.pop(j)'''

two_verb_count = 0

for i in two_verb:
    two_verb_count = two_verb_count + two_verb[i]

two_verb = sorted(two_verb.items(), key=lambda d:d[1], reverse = True)

print (error_num) #0
print (much) #14
print (len(two_verb)) #160
print (two_verb_count) #561
print (skip) #18031
print (index_line)

for i in two_verb:
    print (i[0],i[1])

0
14
160
561
17989
162873
('要(ADV)', '能(ADV)') 41
('當(P)', '可(ADV)') 40
('可(ADV)', '得(Vt)') 29
('要(ADV)', '可(ADV)') 21
('該(ADV)', '會(ADV)') 16
('該(DET)', '會(N)') 16
('要(ADV)', '要(ADV)') 14
('可(ADV)', '得(T)') 12
('要(ADV)', '會(ADV)') 12
('會(ADV)', '要(ADV)') 11
('應(ADV)', '可(ADV)') 11
('可以(ADV)', '得(T)') 10
('該(DET)', '可(ADV)') 9
('必須(ADV)', '可(ADV)') 8
('須(ADV)', '可(ADV)') 8
('要(ADV)', '該(DET)') 8
('當(ADV)', '可(ADV)') 8
('必須(ADV)', '能(ADV)') 8
('可(ADV)', '可(ADV)') 7
('得(T)', '要(ADV)') 7
('該(DET)', '要(ADV)') 7
('會(ADV)', '會(N)') 6
('要(ADV)', '應(ADV)') 6
('必須(ADV)', '要(ADV)') 6
('會(ADV)', '可以(ADV)') 5
('要(ADV)', '得(T)') 5
('會(ADV)', '會(ADV)') 5
('要(ADV)', '可以(ADV)') 5
('能(ADV)', '能(ADV)') 5
('應(ADV)', '要(ADV)') 5
('要(Vt)', '要(ADV)') 5
('該(DET)', '得(T)') 5
('要(ADV)', '能(Vt)') 5
('可以(ADV)', '得(Vt)') 4
('該(DET)', '會(ADV)') 4
('要(ADV)', '得(Vt)') 4
('應(ADV)', '應(ADV)') 4
('應(ADV)', '會(ADV)') 3
('會(N)', '會(ADV)') 3
('該(DET)', '可以(ADV)') 3
('能(ADV)', '得(T)') 3
('會(ADV)', '得(T)') 3
('要(ADV)', '要(V

In [1]:
#找出符合要求的句子(重複句沒抓) one
import xml.etree.ElementTree as ET
import os
import time
from colorama import init

modalverb = ["應","要","可","能","可以","須","應該","必須","會","得","需要","當","應當","能夠","該","需"]
file_path = "C:/Users/user/Desktop/temp/"
file_list = []

for file in os.listdir(file_path):
    file_list.append(file)

find = '可(ADV)'   
    
for file in file_list:
    tree = ET.parse(file_path+file)
    root = tree.getroot()

    for child in root:
        for sentence in child[8]:
            if len(sentence.attrib) == 0:
                temp = sentence.text.split()
                
                lis = []
                
                if find in temp:
                    for ch in temp:
                        if ch == find:
                            lis.append('\033[31;46m' + ch + '\033[0m') 
                        else:
                            lis.append(ch)
                    print (''.join(lis))

                '''for i in temp:

                    word = i.split('(')
                    word[1] = '('+word[1].split('[')[0]

                    if word[0] in modalverb:
                        verb_list.append(''.join(word))
                        verb_index.append(vi)

                    vi = vi + 1
                    
                if len(verb_list) == 2:
                    if verb_list[0] == find[0] and verb_list[1] == find[1]:
                        print (''.join(sentence.text.split()))'''
        time.sleep(0.01)

並(C)謂(Vt)可(ADV)候岳軍(N)回來(Vi)再(ADV)說(Vt)，(COMMACATEGORY)
樊(b)、(PAUSECATEGORY)胡(N)主張(Vt)可(ADV)退(Vi)而(C)不能(ADV)登報(Vi)使(Vt)其(DET)崩潰(Vi)。(PERIODCATEGORY)
而(C)張(N)則(ADV)謂(Vt)此(DET)機關(N)可(ADV)設立(Vt)，(COMMACATEGORY)
但(C)副總統(N)可(ADV)自由(Vi)競選(Vt)，(COMMACATEGORY)
並(C)云(Vt)今日(N)之(T)共黨(N)遠(Vi)非(Vt)民(N)元(N)時(POST)之(T)袁世凱(N)可(ADV)比(P)云(Vt)，(COMMACATEGORY)
只(ADV)有(Vt)哲生(N)主持(Vt)始(ADV)可(ADV)安定(Vt)，(COMMACATEGORY)
依(P)資望(N)或(C)見解(N)((PARENTHESISCATEGORY)在(P)已(ADV)選出(Vt)之(T)立委(N)中(POST))(PARENTHESISCATEGORY)只(ADV)有(Vt)哲(N)生(Vt)一(DET)人(N)可(ADV)任(Vt)，(COMMACATEGORY)
亦(ADV)可(ADV)副北(N)人(N)之(T)希望(Vt)也(T)。(PERIODCATEGORY)
則(ADV)可(ADV)選(Vt)吳鐵城任(N)之(T)，(COMMACATEGORY)
悉(Vt)民社黨(N)決定(Vt)下(DET)週二(N)該(DET)黨(N)立委(N)可(ADV)報到(Vi)也(T)，(COMMACATEGORY)
蔣(N)、(PAUSECATEGORY)孫(N)二(DET)君告(N)以(P)民社黨(N)立委(N)今晚(N)可(ADV)坐車(Vi)來(POST)京報(N)到(Vt)，(COMMACATEGORY)
深(Vi)可(ADV)欽佩(Vt)與(C)歡迎(Vt)。(PERIODCATEGORY)
下午(N)蔡(N)可(ADV)成(Vt)夫人(N)由(P)杭(N)來(Vi)京(N)，(COMMACATEGORY)
僅(ADV)可(ADV)派(Vt)人(N)去(ADV)疏通(Vt)。(PERIODCATEGORY)
可(ADV)提(Vt)楊永浚(N)，(

KeyboardInterrupt: 

In [4]:
#找出符合要求的句子(重複句沒抓) two
import xml.etree.ElementTree as ET
import os
import time

modalverb = ["應","要","可","能","可以","須","應該","必須","會","得","需要","當","應當","能夠","該","需"]
file_path = "C:\\Users\\user\\Desktop\\SNA\\meeting\\source\\diary\\"
file_list = []

for file in os.listdir(file_path):
    file_list.append(file)

find = ['該(DET)', '會(N)']    
    
for file in file_list:
    tree = ET.parse(file_path+file)
    root = tree.getroot()

    for child in root:
        for sentence in child[8]:
            if len(sentence.attrib) == 0:
                temp = sentence.text.split()
                
                verb_list = []  #情態動詞陣列
                verb_index = []  #情態動詞所在位置陣列
                
                vi = 0 #index計算

                for i in temp:

                    word = i.split('(')
                    word[1] = '('+word[1].split('[')[0]

                    if word[0] in modalverb:
                        verb_list.append(''.join(word))
                        verb_index.append(vi)

                    vi = vi + 1
                    
                if len(verb_list) == 2:
                    if verb_list[0] == find[0] and verb_list[1] == find[1]:
                        print (''.join(sentence.text.split()))


故(C)盼(Vt)我(N)任(Vt)該(DET)會(N)之(T)秘書長(N)，(COMMACATEGORY)
予(ADV)係(Vt)該(DET)會(N)主任(N)委員(N)，(COMMACATEGORY)
故(C)今日(N)會議(N)請(Vt)該(DET)會(N)秘書長(N)洪蘭友(N)出席(Vt)。(PERIODCATEGORY)
據(P)勻田云(Vi)在(P)該(DET)黨(N)中委(N)談話(N)會(N)中(POST)，(COMMACATEGORY)
決定(Vt)本(DET)月(N)底(POST)結束(Vt)該(DET)會(N)，(COMMACATEGORY)
謂(Vt)該(DET)會(N)份子(N)複雜(Vi)，(COMMACATEGORY)
渠(N)囑(Vt)我(N)做(Vt)該(DET)會(N)之(T)秘書長(N)。(PERIODCATEGORY)
他(N)接到(Vt)該(DET)會(N)總幹事(N)陳魯仲(N)一(DET)張(M)油印品(N)，(COMMACATEGORY)
該(DET)會(N)欠款(N)三十萬(DET)元(M)，(COMMACATEGORY)


KeyboardInterrupt: 

In [21]:
#雷震日記內容匯出及抓取
import xlrd
import codecs
import time

file = "C:/Users/user/Desktop/雷震日記全表單-資訊處理-20160329.xls"
book = xlrd.open_workbook(file)

sh = book.sheet_by_index(0)

num = 0

with codecs.open("C:/Users/user/Desktop/雷震日記.txt",'w','utf8') as f:
    for rx in range(sh.nrows):
        if rx == 0:
            continue
        content = sh.cell_value(rowx=rx, colx=8)
        num = num + len(''.join(content.strip().split())) #1856456/1868713/1810901 strip/no_strip/strip&split 
        f.write(''.join(content.strip().split())+'\r\n')
        f.write('\r\n')
print (num)

1810901


In [2]:
#修正雷震日記XML部分斷詞失敗問題
from xml.etree import ElementTree as ET
from xml.etree.ElementTree import Element, SubElement, ElementTree
import xlrd
import time
import os
import codecs

from ckip import CKIPSegmenter, CKIPParser
segmenter = CKIPSegmenter('104753018', 'sayanouta')

file = "C:/Users/user/Desktop/雷震日記全表單-資訊處理-20160329.xls"
input_file = "C:/Users/user/Desktop/diary/"
book = xlrd.open_workbook(file)
sh = book.sheet_by_index(0)

file_list = []

for file in os.listdir(input_file):
    file_list.append(file)

with codecs.open("C:/Users/user/Desktop/new.txt",'w','utf8') as f:
    
    for file in file_list:

        tree = ET.parse(input_file+file)
        root = tree.getroot()

        for article in root:

            index = int(article.get('no'))
            content = sh.cell_value(rowx=index, colx=8)

            lines = []
            temp = ''

            for word in content:
                temp = temp + word
                if word == '。' or word == '，':
                    lines.append(temp.strip())
                    temp = ''

            num = len(article[8])

            if len(lines) != num:
                print (file,index,len(lines),num)
                
                f.write(file+' no.'+str(index)+'\r\n')

                for line in lines:
                    
                    try:
                        segmented_result = segmenter.process(line)

                        words = []
                        check = False

                        if segmented_result['status_code'] != '0':
                                print ('Process Failed: ' + segmented_result['status'])
                        else:
                            for sentences in segmented_result['result']:
                                for term in sentences:
                                    if term['pos'] == 'QUESTIONCATEGORY':
                                        check = True
                                    words.append(term['term']+'('+term['pos']+')')

                            if check:
                                f.write('古字：'+line+'\r\n')
                            else:
                                f.write(' '.join(words)+'\r\n')
                    except:
                        f.write('斷詞失敗：'+line+'\r\n')
                
                f.write('\r\n')
    print ('END')

END


In [1]:
import xml.etree.ElementTree as ET
import os
import time
import codecs
from ipywidgets import IntProgress
from IPython.display import display

p = IntProgress()
p.max = 16
p.description = 'start'
display(p)

#modalverb = ["應","要","可","能","可以","須","應該","必須","會","得","需要","當","應當","能夠","該","需"]
modalverb = ["應","要","可","能","可以","須","應該","必須","會","得","需要","當","應當","能夠","該","需"]
file_path = "C:\\Users\\user\\Desktop\\SNA\\meeting\\source\\diary\\"
out_path = "C:\\Users\\user\\Desktop\\diary_word\\"
file_list = []
count = 0

for file in os.listdir(file_path):
    file_list.append(file)

for find in modalverb: 

    dic = {}
    sen = {}

    for file in file_list:
        tree = ET.parse(file_path+file)
        root = tree.getroot()

        for child in root:
            for sentence in child[8]:
                if len(sentence.attrib) == 0:
                    temp = sentence.text.split()

                    index = 0
                    pairs = []

                    for word in temp:

                        words = word.split('(')
                        words[1] = '('+words[1]

                        if words[0] == find:
                            '''if index != 0:
                                pairs.append((index-1,index))'''
                            if index != len(temp)-1:
                                if temp[index+1].split('(')[1][0] != 'V':
                                    if temp[index+1].split('(')[1][:-1] != 'COMMACATEGORY' and  \
                                        temp[index+1].split('(')[1][:-1] != 'PERIODCATEGORY':
                                        pairs.append((index,index+1))

                        index += 1

                    for i in pairs:
                        if temp[i[0]]+temp[i[1]] not in dic:
                            dic[temp[i[0]]+temp[i[1]]] = 1
                        else:
                            dic[temp[i[0]]+temp[i[1]]] += 1
                        
                        if temp[i[0]]+temp[i[1]] not in sen:
                            sen[temp[i[0]]+temp[i[1]]] = [''.join(temp)]
                        elif len(sen[temp[i[0]]+temp[i[1]]]) < 3:
                            sen[temp[i[0]]+temp[i[1]]].append(''.join(temp))

    dic = sorted(dic.items(), key=lambda d:d[1], reverse = True)
    with codecs.open(out_path+find+'.txt','wb','utf8') as f:
        for i in dic:
            #print (i[0],i[1])
            f.write(str(i[0])+str(i[1])+'\r\n')
            for j in sen[i[0]]:
                f.write(j+'\r\n')
            f.write('\r\n')
            
    count = count + 1
    p.value = count
    p.description = str(count)
p.description = 'end'

In [2]:
import xml.etree.ElementTree as ET
import os
import time
import codecs
from ipywidgets import IntProgress
from IPython.display import display

p = IntProgress()
p.max = 945
p.description = 'start'
display(p)

modalverb = ["應","要","可","能","可以","須","應該","必須","會","得","需要","當","應當","能夠","該","需"]
file_path = "C:\\Users\\user\\Desktop\\SNA\\meeting\\source\\diary\\"
out_path = "C:\\Users\\user\\Desktop\\"
file_list = []
count = 0

for file in os.listdir(file_path):
    file_list.append(file)

dic = {}
sen = {}

for file in file_list:
    tree = ET.parse(file_path+file)
    root = tree.getroot()

    for child in root:
        for sentence in child[8]:
            if len(sentence.attrib) == 0:
                temp = sentence.text.split()

                index = 0
                pairs = []

                for word in temp:

                    words = word.split('(')
                    words[1] = '('+words[1]

                    if words[0] in modalverb:
                        '''if index != 0:
                            pairs.append((index-1,index))'''
                        if index != len(temp)-1:
                            if (temp[index+1].split('(')[1][0] == 'V' or temp[index+1].split('(')[1][0] == 'A' or \
                               temp[index+1].split('(')[1][0] == 'P') and (words[1] == '(V)' or words[1] == '(ADV)'):
                                if temp[index+1].split('(')[1][:-1] != 'COMMACATEGORY' and  \
                                    temp[index+1].split('(')[1][:-1] != 'PERIODCATEGORY':
                                    pairs.append(temp[index])

                    index += 1

                if len(pairs) > 1:
                        
                    for i in range(len(pairs)-1):    
                        
                        term = pairs[i]+'_'+pairs[i+1]

                        if term not in dic:
                            dic[term] = 1
                        else:
                            dic[term] += 1

                        if term not in sen:
                            sen[term] = [''.join(temp)]
                        elif len(sen[term]) < 3:
                            sen[term].append(''.join(temp))
    count = count + 1
    p.value = count
    p.description = str(count)

dic = sorted(dic.items(), key=lambda d:d[1], reverse = True)
with codecs.open(out_path+'output.txt','wb','utf8') as f:
    for i in dic:
        #print (i[0],i[1])
        f.write(str(i[0])+' '+str(i[1]))
        '''for j in sen[i[0]]:
            f.write(j+'\r\n')'''
        f.write('\r\n')
        
p.description = 'end'